In [1]:
# Import Statements
import pyrealsense2 as rs
import numpy as np
import math
import time
import cv2
import copy

In [2]:
class rotation_estimator:
    def __init__(self):
        self.theta = [0.0,0.0,0.0]
        self.alpha = 0.98
        self.first = True
        self.last_ts_gyro = 0.0
    
    # Function to calculate the change in angle of motion based on data from gyro
    # gyro_data - list of 3 elems
    # ts - float arrival time of curr gyro frame
    def process_gyro(self, gyro_data, ts):
        if self.first:
            self.last_ts_gyro = ts
            return
        
        gyro_angle = [gyro_data.x,gyro_data.y,gyro_data.z]
        
        dt_gyro = (ts-self.last_ts_gyro)/1000.0
        self.last_ts_gyro = ts
        
        gyro_angle = [gyro_angle[0]*dt_gyro,gyro_angle[1]*dt_gyro,gyro_angle[2]*dt_gyro]
        
        self.theta = [self.theta[0]-gyro_angle[2],self.theta[1]-gyro_angle[1],self.theta[2]+gyro_angle[0]]
    
    # Function to calculate the change in angle of motion based on data from accelerometer
    # accel_data - list of 3 elems
    def process_accel(self, accel_data):
        accel_angle = [0.0,0.0,0.0]
        accel_angle[2] = np.arctan2(accel_data.y,accel_data.z)
        accel_angle[0] = np.arctan2(accel_data.x, np.sqrt(accel_data.y**2+accel_data.z**2))
        
        if self.first:
            self.first = False
            self.theta = accel_angle
#             self.theta[1] = np.pi
            self.theta[1] = 0.0
        else:
            self.theta[0] = self.theta[0]*self.alpha + accel_angle[0]*(1-self.alpha)
            self.theta[2] = self.theta[2]*self.alpha + accel_angle[2]*(1-self.alpha)
        
    def get_theta(self):
        theta_out = [0.0,0.0,0.0]
        theta_out = [self.theta[0]*180/np.pi, self.theta[1]*180/np.pi, (self.theta[2]-np.pi/2)*180/np.pi]
        return theta_out

In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.gyro)
config.enable_stream(rs.stream.accel)

In [4]:
algo = rotation_estimator()
profile = pipeline.start(config)

try:
    while True:
        frames = pipeline.wait_for_frames()
        
        ts = frames[1].as_motion_frame().get_timestamp()
        gyro_data = frames[1].as_motion_frame().get_motion_data()
        algo.process_gyro(gyro_data, ts)
#         print("Gyro: ", gyro_sample.x, " ", gyro_sample.y, " ", gyro_sample.z, "\n")
        
        accel_data = frames[0].as_motion_frame().get_motion_data()
        algo.process_accel(accel_data)
#         print("Accel: ", accel_sample.x, " ", accel_sample.y, " ", accel_sample.z, "\n")

        out = algo.get_theta()
        print(out)
#         print('x: ', out[0], ' y: ', out[1], ' z: ', out[2])
        
    
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        
finally:

    # Stop streaming
    pipeline.stop()

[3.926651139965928, 0.0, -188.41637430265564]
[4.038057934632294, -0.08981982671821433, -188.4632671444515]
[4.055818617384405, -0.09980078367214817, -188.5008397113129]
[3.9679207610940814, -0.08982068121039521, -188.53301662694722]
[3.9118363116720585, -0.11375974352495447, -188.55799402842464]
[3.9332178893333234, -0.17750114928119015, -188.58190400425582]
[3.9748555443347477, -0.23125461672401587, -188.57974925828088]
[3.92536409787615, -0.24319118889717578, -188.5720761568371]
[3.7931453931057884, -0.25114480211681417, -188.53967235508927]
[3.6576729606370613, -0.2670403098061817, -188.50311538278217]
[3.6100214489344884, -0.2725045922554875, -188.5080241177363]
[3.567934812328889, -0.2764762719122214, -188.51598992919958]
[3.5437196887393285, -0.2764762719122214, -188.53153631790454]
[3.507774273727795, -0.2804475609439583, -188.54029916529353]
[3.492953154523267, -0.2804475609439583, -188.55814587897484]
[3.4808303142468846, -0.2804475609439583, -188.57563565838248]
[3.458041116

[2.9330990356790756, -0.3313308881261948, -187.84231088829213]
[2.9337874651892166, -0.3343112591968577, -187.84628559784426]
[2.937158649822417, -0.3343112591968577, -187.84626900319222]
[2.942307643861595, -0.3343112591968577, -187.8370854660062]
[2.9387554880113, -0.3348080121226972, -187.82597918505908]
[2.9452516574752203, -0.3348080121226972, -187.82006466862435]
[2.942686120083743, -0.3338145062710183, -187.81329480678372]
[2.9479182730678577, -0.3338145062710183, -187.81331253240631]
[2.948340079674366, -0.3338145062710183, -187.8072020002764]
[2.942963371256869, -0.332324101009126, -187.79776544477937]
[2.9458108729518018, -0.332324101009126, -187.78516171855946]
[2.942203558797847, -0.33133049750119786, -187.7738011283203]
[2.946961961457606, -0.33133049750119786, -187.7619535776637]
[2.9514460759884322, -0.33133049750119786, -187.74879297266193]
[2.9475501738227528, -0.32834939400866553, -187.7351393270638]
[2.9502601831753386, -0.32834939400866553, -187.72137741645957]
[2.9

[2.4913370405597615, -0.4713330161016411, -187.7489647325644]
[2.5265796941251906, -0.47331905124250656, -187.72505391783545]
[2.561020942456289, -0.47977334780258085, -187.69506394215654]
[2.5577507053439312, -0.47977334780258085, -187.6652557398738]
[2.5518801457154265, -0.47977334780258085, -187.6348493380932]
[2.584056404005379, -0.48821731719868766, -187.61188635606422]
[2.5793914045427875, -0.48821731719868766, -187.58681490359504]
[2.6086368686125128, -0.49715403534960406, -187.55867065496318]
[2.6086703551049375, -0.49715403534960406, -187.53838761181964]
[2.606077503795583, -0.49715403534960406, -187.52383282847015]
[2.6375626898547715, -0.5055942693942946, -187.5034252454371]
[2.638782883246209, -0.5055942693942946, -187.48326214678772]
[2.6548263055031414, -0.511552374816891, -187.45616973289592]
[2.655615587649547, -0.511552374816891, -187.44325930712492]
[2.663418335747467, -0.5135383123015071, -187.42370089541944]
[2.6686866078161926, -0.5135383123015071, -187.40897992864

[2.619424960212849, -0.44620684177349296, -187.85800173601504]
[2.5987561876783714, -0.44868657808240897, -187.90801201958834]
[2.5941790795959014, -0.44868657808240897, -187.9404293584064]
[2.572868651054222, -0.44868657808240897, -187.98225148517875]
[2.5689905282369687, -0.44868657808240897, -188.01509802835815]
[2.5671694438085892, -0.44868657808240897, -188.0377859218294]
[2.55586553520966, -0.44571206638670063, -188.06901572617303]
[2.5517257744008788, -0.44571206638670063, -188.09650491512025]
[2.5418543718895035, -0.4392676086933337, -188.12944060545848]
[2.5379864295135746, -0.4392676086933337, -188.14987971495688]
[2.534261255718722, -0.4392676086933337, -188.15971094175765]
[2.527372239138106, -0.42588296497780415, -188.1764054452728]
[2.518527983910216, -0.41001968385068693, -188.18325221155357]
[2.507877386480126, -0.41001968385068693, -188.1756063233564]
[2.4982987134975256, -0.3941501527236181, -188.16332416569725]
[2.4927213252040024, -0.3941501527236181, -188.147360281

[2.4774269912684903, 0.09541144202241261, -188.4579867356885]
[2.444745999856428, 0.11035211626629973, -188.4706413423422]
[2.4332749946231633, 0.11035211626629973, -188.46085244522024]
[2.4058970862364566, 0.1252993823072453, -188.46706606133176]
[2.39900016337638, 0.1252993823072453, -188.45849385409286]
[2.3922949502281385, 0.1252993823072453, -188.44792167686842]
[2.3687090424178954, 0.13625641400389224, -188.4452710501194]
[2.367529704696185, 0.13625641400389224, -188.4320959438183]
[2.337744885604587, 0.14472321054754325, -188.4281463127446]
[2.326693472228409, 0.14472321054754325, -188.41242029699546]
[2.321663374525298, 0.14472321054754325, -188.4000629337847]
[2.309258553635859, 0.14970392339958583, -188.39629550344625]
[2.304588714772084, 0.14970392339958583, -188.38190922331765]
[2.2999697466809814, 0.14970392339958583, -188.36765230645216]
[2.29035200651096, 0.15020201909885242, -188.35586383406513]
[2.2822683544296987, 0.15020201909885242, -188.35084827228297]
[2.280865476

...

[-0.5103553568626847, 2.3570490833878845, -171.33347157960895]
[-0.5089185502104653, 2.3570490833878845, -171.31737547207973]
[-0.5142067697034721, 2.3570490833878845, -171.30123662378278]
[-0.5453425060022691, 2.37448841457336, -171.26374302173036]
[-0.5533498178315347, 2.37448841457336, -171.23714834811733]
[-0.5882173674997304, 2.3894356806143056, -171.20002228337358]
[-0.591028794882002, 2.3894356806143056, -171.17313419595163]
[-0.601711387000841, 2.3894356806143056, -171.1380388137304]
[-0.6322983893893903, 2.4038847036740116, -171.08739872545652]
[-0.6422572571640783, 2.4038847036740116, -171.04499430163753]
[-0.6765207652607154, 2.417343175515428, -170.9856880783479]
[-0.6834296767203705, 2.417343175515428, -170.94890715032778]
[-0.7005182797957382, 2.417343175515428, -170.90864649668592]
[-0.7302984984223718, 2.4273080191882372, -170.85145615893762]
[-0.744029030939079, 2.4273080191882372, -170.80733903635502]
[-0.7794010577494095, 2.435279894126484, -170.7510010559137]
[-0.79

[-2.481827339427822, 2.1121209838990844, -169.27950656500414]
[-2.478210974603591, 2.1121209838990844, -169.28842219619244]
[-2.4801456500550394, 2.1121209838990844, -169.29425217534626]
[-2.470584345679113, 2.127085095643624, -169.31218624547938]
[-2.4694984830625213, 2.127085095643624, -169.3224606954486]
[-2.46354606136894, 2.1405527960007764, -169.34132855328954]
[-2.462694386763749, 2.1405527960007764, -169.35062211806508]
[-2.452184269737563, 2.149531262195769, -169.36817133078125]
[-2.447067967123546, 2.149531262195769, -169.37803714448827]
[-2.4415906491544077, 2.1570162477556205, -169.40425869140847]
[-2.43691034860833, 2.1570162477556205, -169.42729949959178]
[-2.427805513690828, 2.1570162477556205, -169.4509969707003]
[-2.4102860018775973, 2.16449830362789, -169.4760359790954]
[-2.398336249648914, 2.16449830362789, -169.50211331278223]
[-2.37011548911133, 2.1709824341708837, -169.53361326898445]
[-2.3454550993694823, 2.1709824341708837, -169.56136005398437]
[-2.3093933399205

[-1.6523021978590615, 2.2652651721658077, -168.96298077635657]
[-1.6625681997579584, 2.2652651721658077, -168.94902419863976]
[-1.6270901416807633, 2.253293492571045, -168.92840287541082]
[-1.621195013766633, 2.2473084340236573, -168.9205073773962]
[-1.6310825498900274, 2.2473084340236573, -168.9268929273875]
[-1.6238743819806225, 2.2433168324920776, -168.92611232704613]
[-1.6336870256014282, 2.2433168324920776, -168.9417999789711]
[-1.6352558824641505, 2.2433168324920776, -168.95400761581797]
[-1.6131089160108139, 2.240324303218384, -168.97067003682463]
[-1.6092300708674436, 2.240324303218384, -168.99267579499693]
[-1.5931708865202587, 2.239326793460486, -169.0143813998572]
[-1.5933302023328426, 2.239326793460486, -169.03751265935082]
[-1.582748146414447, 2.24231932273418, -169.05953548882323]
[-1.5832106398549455, 2.24231932273418, -169.08117607668942]
[-1.5837137997908646, 2.24231932273418, -169.0927361499219]
[-1.5723595143151072, 2.247308824648654, -169.11343313256356]
[-1.5731229

[-1.7846699289811414, 2.1984317248858023, -169.09588626276806]
[-1.7809885673291848, 2.1984317248858023, -169.07319158227702]
[-1.775803756540594, 2.2133994987398187, -169.04465805481803]
[-1.7723703097051355, 2.2133994987398187, -169.01306442995022]
[-1.769085621143219, 2.2133994987398187, -168.9791114088454]
[-1.7647185276253528, 2.2258775257205734, -168.9507288348435]
[-1.7616599337722076, 2.2258775257205734, -168.91389346387436]
[-1.7585816083742207, 2.2343621689432136, -168.87995647298376]
[-1.7555938016897072, 2.2343621689432136, -168.8520306022381]
[-1.7549514134239603, 2.2343621689432136, -168.81768202057262]
[-1.753088367720833, 2.240353965771799, -168.78736686587152]
[-1.7492032441592642, 2.240353965771799, -168.75302571848712]
[-1.7495064822124966, 2.2418513290414497, -168.7212833509156]
[-1.751499065442516, 2.2418513290414497, -168.6880051883727]
[-1.756193426049786, 2.239355845662343, -168.6553023548244]
[-1.7538159259409356, 2.239355845662343, -168.63580739844326]
[-1.755

KeyboardInterrupt: 